<a href="https://colab.research.google.com/github/Redcoder815/Deep_Learning_PyTorch/blob/main/TypesOfRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

One-to-One RNN


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim

# ----- Model Definition -----
class SimpleRNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = nn.RNN(
            input_size=10,   # matches last dimension of X
            hidden_size=32,
            batch_first=True
        )
        self.fc = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out, _ = self.rnn(x)        # out: (batch, seq_len, hidden)
        out = out[:, -1, :]         # take last timestep
        out = self.fc(out)
        return self.sigmoid(out)

# ----- Data -----
X = torch.rand(1000, 1, 10)        # same shape as TF: (batch, seq, features)
y = torch.randint(0, 2, (1000, 1)).float()

# ----- Model, Loss, Optimizer -----
model = SimpleRNNModel()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

# ----- Training Loop -----
epochs = 10
batch_size = 32

for epoch in range(epochs):
    permutation = torch.randperm(X.size(0))

    for i in range(0, X.size(0), batch_size):
        idx = permutation[i:i+batch_size]
        batch_x = X[idx]
        batch_y = y[idx]

        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    # optional: print progress
    # print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

One-to-Many RNN


In [21]:
import torch
import torch.nn as nn
import torch.optim as optim

# One-to-Many RNN model in PyTorch
class OneToManyRNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(2048, 64)          # Dense layer
        self.repeat = 20                       # RepeatVector(20)
        self.rnn = nn.RNN(
            input_size=64,
            hidden_size=128,
            batch_first=True,
            nonlinearity='tanh'
        )
        self.out = nn.Linear(128, 10000)       # Output layer

    def forward(self, x):
        # x: (batch, 2048)
        x = self.fc(x)                         # (batch, 64)
        x = x.unsqueeze(1).repeat(1, self.repeat, 1)  # (batch, 20, 64)
        x, _ = self.rnn(x)                     # (batch, 20, 128)
        x = self.out(x)                        # (batch, 20, 10000)
        # return torch.softmax(x, dim=-1)
        return x


# Instantiate model
model = OneToManyRNN()

# Loss + optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Simulated data
X = torch.rand(500, 2048)
y = torch.rand(500, 20, 10000)

# PyTorch expects class indices for CrossEntropyLoss,
# but your TF code uses one-hot softmax targets.
# So we convert to class indices:
y_indices = torch.argmax(y, dim=-1)   # (500, 20)

# Training loop
epochs = 5
batch_size = 16

for epoch in range(epochs):
    for i in range(0, len(X), batch_size):
        xb = X[i:i+batch_size]
        yb = y_indices[i:i+batch_size]

        optimizer.zero_grad()
        preds = model(xb)                     # (batch, 20, 10000)

        # CrossEntropyLoss expects (batch*seq, classes)
        loss = criterion(
            preds.reshape(-1, 10000),
            yb.reshape(-1)
        )

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, loss={loss.item():.4f}")

Epoch 1, loss=9.2566
Epoch 2, loss=9.3040
Epoch 3, loss=8.4692
Epoch 4, loss=8.1023
Epoch 5, loss=7.9306


Many-to-One RNN


In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# -----------------------------
# Model using your forward()
# -----------------------------
class ManyToOneRNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = nn.RNN(
            input_size=128,
            hidden_size=64,
            batch_first=True
        )
        self.fc = nn.Linear(64, 32)     # Dense(32, relu)
        self.relu = nn.ReLU()
        self.out1 = nn.Linear(32, 3)    # Dense(3, softmax) → logits only

    def forward(self, x):
        out, _ = self.rnn(x)            # (batch, seq, hidden)
        out = out[:, -1, :]             # last timestep
        out = self.fc(out)
        out = self.relu(out)
        out = self.out1(out)            # raw logits
        return out                      # CrossEntropyLoss expects logits

# -----------------------------
# Random data like your TF code
# -----------------------------
X = np.random.rand(2000, 100, 128).astype(np.float32)
y = np.random.randint(0, 3, 2000)

X = torch.tensor(X)
y = torch.tensor(y)

# -----------------------------
# Training setup
# -----------------------------
model = ManyToOneRNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# -----------------------------
# Training loop
# -----------------------------
batch_size = 32
epochs = 10

for epoch in range(epochs):
    permutation = torch.randperm(X.size(0))

    for i in range(0, X.size(0), batch_size):
        idx = permutation[i:i+batch_size]
        batch_x = X[idx]
        batch_y = y[idx]

        optimizer.zero_grad()
        logits = model(batch_x)
        loss = criterion(logits, batch_y)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{epochs} - Loss: {loss.item():.4f}")

Epoch 1/10 - Loss: 1.1063
Epoch 2/10 - Loss: 1.0941
Epoch 3/10 - Loss: 1.0934
Epoch 4/10 - Loss: 1.1029
Epoch 5/10 - Loss: 1.0876
Epoch 6/10 - Loss: 1.0976
Epoch 7/10 - Loss: 1.0678
Epoch 8/10 - Loss: 1.0533
Epoch 9/10 - Loss: 0.9004
Epoch 10/10 - Loss: 0.8767


Many-to-Many RNN

In [22]:
import torch
import torch.nn as nn

class ManyToManyRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super().__init__()
        self.rnn = nn.RNN(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, h0=None):
        # x: (batch, seq_len, input_size)
        out, hn = self.rnn(x, h0)
        # out: (batch, seq_len, hidden_size)
        logits = self.fc(out)
        # logits: (batch, seq_len, output_size)
        return logits, hn


# -----------------------------
# Example usage
# -----------------------------
batch_size = 4
seq_len = 6
input_size = 10
hidden_size = 32
output_size = 8

model = ManyToManyRNN(input_size, hidden_size, output_size)

# Fake data
x = torch.randn(batch_size, seq_len, input_size)
y = torch.randn(batch_size, seq_len, output_size)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for step in range(200):
    optimizer.zero_grad()
    logits, _ = model(x)
    loss = criterion(logits, y)
    loss.backward()
    optimizer.step()

    if step % 50 == 0:
        print(f"step {step}, loss = {loss.item():.4f}")

step 0, loss = 0.7726
step 50, loss = 0.4434
step 100, loss = 0.1895
step 150, loss = 0.0658
